# 81. Gradio_Stream

## Overview  
This exercise demonstrates how to build a Retrieval-Augmented Generation (RAG) system using Gradio and how to generate and stream responses in real-time using its streaming features. Through this exercise, you will learn to handle real-time interactions with users via a web-based interface. This process helps manage the overall conversation flow, thereby providing more detailed and meaningful responses.
 
## Purpose of the Exercise
The purpose of this exercise is to implement real-time response generation and streaming capabilities using Gradio to develop a live interactive chatbot interface. By the end of this tutorial, users will be able to create a dynamic chat system that streams responses as they are generated, enhancing user engagement and interaction.



In [2]:
!pip install -qU gradio python-dotenv langchain-upstage python-dotenv

In [10]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [39]:
import gradio as gr

from langchain_upstage import UpstageDocumentParseLoader, UpstageGroundednessCheck, ChatUpstage, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage

from tokenizers import Tokenizer

llm = ChatUpstage(streaming=True)

In [21]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

In [23]:
doc_loader = UpstageDocumentParseLoader("laws-of-the-game-2024-25-korean-en.pdf", output_format='html', coordinates=False)
docs = doc_loader.load()

In [25]:
for doc in docs:
    pprint(doc.page_content[:100])

("<h1 id='0' style='font-size:18px'>경기규칙</h1><br><h1 id='1' "
 "style='font-size:16px'>Laws of the Game</h")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, output_formats="text")
splits = text_splitter.split_documents(docs)

print("Splits:", len(splits)) 

Splits: 235


In [ ]:
# semantic chunking split

def semantic_chunker(docs, min_chunk_size=100, chunk_overlap=10, max_chunk_size=1000, merge_threshold=0.7, embeddings=UpstageEmbeddings(model="solar-embedding-1-large")):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=min_chunk_size, chunk_overlap=chunk_overlap)
    init_splits = text_splitter.split_documents(docs)
    splits = []

    base_split_text = None
    base_split_emb = None
    for split in init_splits:
        if base_split_text is None:
            base_split_text = split.page_content
            base_split_emb = embeddings.embed_documents([base_split_text])[0]
            continue

        split_emb = embeddings.embed_documents([split.page_content])[0]
        distance = cosine_similarity(X=[base_split_emb], Y=[split_emb])
        if (distance[0][0] < merge_threshold or len(base_split_text) + len(split.page_content) > max_chunk_size):
            splits.append(Document(page_content=base_split_text))
            base_split_text = split.page_content
            base_split_emb = split_emb
        else:
            base_split_text += split.page_content

    if base_split_text:
        splits.append(Document(page_content=base_split_text))

    return splits

방방곡곡 남녀노소의 사람들 덕분에 그 말의 다양성을 채워 왔습니다. 물론,<br>그 다채로운 언어적 배경에도 그들 모두가 그 기준을 표준의 서울말로 삼아 표기의 통일성<br>을 지켰습니다.</p><p id='14' data-category='paragraph' style='font-size:16px'>본 경기규칙서는 대한민국 국립국어원의 어문규정에 충실하고자 노력하였습니다. 또한 표<br>준말의 출처로는 국립국어원의 “우리말 샘” “표준 국어 대사전”과 고려대학교 민족문화연<br>구원의 “고려대 한국어 대사전”, “옥스퍼드 영한사전 제9판”이 사용되었습니다. 그러나 이</p><footer id='15' style='font-size:14px'>4</footer><br><p id='16' data-category='paragraph' style='font-size:14px'>5</p><p id='17' data-category='paragraph' style='font-size:18px'>미 대한민국의 축구 가족들 사이에서 그 사용이 오래되어 굳어진 말이나 외래어의 경우, 또<br>는 보다 분명한 뜻을 전달하고자 영어로 쓰인 표현을 그대로 사용해야 할 경우, 그 원칙을<br>충실히 따르지 못한 부분도 있습니다. 그 점은 이 규칙서를 읽는 분들의 양해를 구합니다.</p><p id='18' data-category='paragraph' style='font-size:18px'>전 세계로 퍼져나간 우리 겨레와 그 헤어진 후손들을 위하여, 그들의 어머니, 혹은 그들의<br>어머니의 어머니가 그들을 키웠던 말로 “축구의 법”을 옮기는 일. 그 소명에 대한축구협회<br>심판위원회는 앞으로도 충실할 것입니다.</p><p id='19' data-category='paragraph' style='font-size:18px'>우리가 옮긴 내용이 충분하지 않거나, 혹은 그 뜻에 바르지 않음이 있다면 아래의 주소로 메<br>일을 보내주시기를 부탁드립니다.</p><h1 id='20'


In [27]:
# hfembeddings = HuggingFaceEmbeddings(model_name="klue/roberta-small")
u_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [ ]:
semantic_splits = semantic_chunker(docs, merge_threshold=0.8, embeddings=u_embeddings)

print("SemanticChunker Splits:", len(semantic_splits))

In [ ]:
vectorstore = Chroma.from_documents(documents=semantic_splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large"))

In [37]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [40]:
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

In [66]:
#
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            너는 축구 용어에 대한 질문에 답하는 AI 챗봇이야.
            제공된 문서를 참고하고, 질문 히스토리에 기반해서 답변해줘.
            답변을 모르면 그냥 모른다고 답해줘.
            ---
            CONTEXT:
            {context}
            """
        ), 
        MessagesPlaceholder(variable_name='history'), 
        ("human", "{input}"),
    ]
)

In [67]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [42]:
def get_relevant_context(question, retriever):
    # retriever를 사용해 관련된 문서만 가져옴
    relevant_docs = retriever.get_relevant_documents(question)
    
    # 관련 문서들을 하나의 문자열로 합침
    return "\n".join(doc.page_content for doc in relevant_docs)


In [62]:
def chat_2(message, history):
    print(message)
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    generator = chain.stream({"message": message, "history": history_langchain_format})

    assistant = ""
    for gen in generator:
        assistant += gen
        yield assistant

In [70]:
def chat(question, history):
    print(question)
    history_langchain_format = []

    #####
    # 기존 질문 및 컨텍스트 처리
    relevant_context = get_relevant_context(question, retriever)
    # 모델 호출
    ai = chain.invoke({
        "history": history_langchain_format, 
        "context": relevant_context, 
        "input": question
    })
    #####

    # for human, ai in history:
    #     history_langchain_format.append(HumanMessage(content=human))
    #     history_langchain_format.append(AIMessage(content=ai))

    #generator = chain.stream({"message": question, "history": history_langchain_format})

    # assistant = ""
    # for gen in generator:
    #     assistant += gen
    #     yield assistant

    return ai

In [74]:
import gradio as gr
from __future__ import annotations
from typing import Iterable
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes

# CSS로 Football 스타일 정의
css = """
/* 전체 배경 설정 */
.gradio-container {
    background-color: #6da682;
}

/* 블록(박스) 설정 및 텍스트에 그림자 추가 */
.gr-block {
    background-color: #ffffff;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    border-radius: 10px; /* 모서리 둥글기 */
    padding: 20px;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.6); /* 모든 텍스트에 그림자 적용 */
}

/* 버튼 스타일 */
button.primary {
    background: linear-gradient(90deg, #66cdaa, #008b8b);
    color: black;
    padding: 12px 24px;
    border-radius: 8px; /* 버튼 모서리 둥글기 */
    box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.2);
}

button.primary:hover {
    background: linear-gradient(90deg, #50a37c, #006b6b);
}

button.primary:active {
    background: linear-gradient(90deg, #006b6b, #66cdaa);
}

/* 슬라이더 색상 */
.gr-slider .track-fill {
    background-color: #66cdaa;
}

/* 입력 필드 배경 */
input[type="text"], textarea {
    background-color: #ffffff;
    border-radius: 8px; /* 입력 필드 모서리 둥글기 */
    padding: 10px;
    box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
}

/* 메시지 창 전체 배경 설정 */
.gr-chatbot {
    background-color: #ffffff; /* 메시지 창 전체 흰색 배경 */
    border: none;
    padding: 0;
}

/* 전체 배경 이미지 설정 */
.gradio-container {
    background-image: url('https://static.vecteezy.com/system/resources/previews/013/950/541/non_2x/football-field-flat-flat-icon-free-vector.jpg');
    background-size: cover;
    background-position: center; /* 이미지 가운데 정렬 */
    background-repeat: no-repeat; /* 이미지 반복하지 않음 */
}

/* 제목 텍스트에 추가적인 그림자 효과 */
#gradio-animation {
    color: white;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.6); /* 그림자 효과 */
}
"""

# JavaScript 코드
js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';

    var text = 'FootBot⚽️';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.5s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);

    return 'Animation created';
}
"""


In [75]:
# with gr.Blocks() as demo:
#     chatbot = gr.ChatInterface(
#         chat,
#         examples=[
#             "How to eat healthy?",
#             "Best Places in Korea",
#             "How to make a chatbot?",
#         ],
#         title="Solar Chatbot",
#         description="Upstage Solar Chatbot",
#     )
#     chatbot.chatbot.height = 300

In [76]:
from langchain_upstage import ChatUpstage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from collections import Counter
import requests
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to fetch news articles from Naver Soccer News API
def fetch_articles(api_url, headers):
    response = requests.get(api_url, headers=headers)
    response.raise_for_status()
    articles = response.json()
    return articles['items'][:100]

# Function to clean and tokenize text
def clean_and_tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.lower().split()
    return tokens

# Function to calculate keyword frequency for importance
def calculate_importance(article_text, common_keywords):
    tokens = clean_and_tokenize(article_text)
    keyword_count = sum(1 for token in tokens if token in common_keywords)
    return keyword_count

# Function to remove duplicate articles based on content similarity
def remove_duplicates(articles, threshold=0.8):
    descriptions = [article['description'] for article in articles]
    vectorizer = TfidfVectorizer().fit_transform(descriptions)
    similarity_matrix = cosine_similarity(vectorizer)
    unique_articles = []
    seen_indices = set()

    for i, article in enumerate(articles):
        if i not in seen_indices:
            unique_articles.append(article)
            similar_indices = np.where(similarity_matrix[i] > threshold)[0]
            seen_indices.update(similar_indices)

    return unique_articles

# Fetch, process, and summarize articles
def summarize_articles():
    api_url = "https://openapi.naver.com/v1/search/news.json?query=축구&display=100&sort=date"
    headers = {
        "X-Naver-Client-Id": "aMevyrYH7wOTUGOb7H7l",
        "X-Naver-Client-Secret": "gSdqbjXchL"
    }

    # Fetch and process articles
    articles = fetch_articles(api_url, headers)
    articles = remove_duplicates(articles)

    # Aggregate text to find common keywords
    all_text = " ".join([article['description'] for article in articles if 'description' in article])
    common_tokens = Counter(clean_and_tokenize(all_text)).most_common(20)
    common_keywords = [token for token, _ in common_tokens]

    # Initialize LLM and summarization chain
    llm = ChatUpstage()
    prompt_template = PromptTemplate(input_variables=["article"], template="다음 기사를 요약해 주세요: {article}")
    summary_chain = LLMChain(llm=llm, prompt=prompt_template)

    # Summarize and rank articles by importance
    important_articles = []
    for article in articles:
        if 'description' in article:
            importance_score = calculate_importance(article['description'], common_keywords)
            summary = summary_chain.run(article['description'])
            title_prompt = f"이 기사에 적합한 제목을 지어주세요: {article['description']}"
            title = llm.generate([title_prompt]).generations[0][0].text.strip()
            important_articles.append((title, summary, importance_score))

    # Sort and return top 3 articles
    important_articles.sort(key=lambda x: x[2], reverse=True)
    return important_articles[:3]

# Gradio Interface
def display_summaries():
    articles = summarize_articles()
    summary_md = "\n\n".join(
        [f"### 🤖 **{title}**\n{summary}" for title, summary, _ in articles]
    )
    return summary_md

In [77]:
with gr.Blocks(js=js, css=css) as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("<h1 style='text-align: center;'>오늘의 축구 뉴스 요약</h1>")
            article_summaries = gr.Markdown(value=display_summaries, elem_id="article_summaries")
        with gr.Column():
            chatbot = gr.ChatInterface(
                chat,
                examples=[
                    "오프사이드는 어떤 상황에서 발생하나요?",
                    "패널티킥은 언제 주어지나요?",
                    "옐로우카드와 레드카드의 차이는 무엇인가요?",
                ],
                title="궁금한 사항을 검색해보세요",
                description="모르는 용어나 규칙을 질문할 수 있어요!",
            )
            chatbot.chatbot.height = 300 

In [79]:
if __name__ == "__main__":
    demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


패널티킥은 언제 주어지나요?
